# 1 EDA and Data Cleaning

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# read in data
data = pd.read_csv('../data/Crime_Data_2010_2017.csv')
data.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Area Name,Reporting District,Crime Code,Crime Code Description,MO Codes,...,Weapon Description,Status Code,Status Description,Crime Code 1,Crime Code 2,Crime Code 3,Crime Code 4,Address,Cross Street,Location
0,1208575,03/14/2013,03/11/2013,1800,12,77th Street,1241,626,INTIMATE PARTNER - SIMPLE ASSAULT,0416 0446 1243 2000,...,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,626.0,NaN,NaN,NaN,6300 BRYNHURST AV,NaN,"(33.9829, -118.3338)"
1,102005556,01/25/2010,01/22/2010,2300,20,Olympic,2071,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,VAN NESS,15TH,"(34.0454, -118.3157)"
2,418,03/19/2013,03/18/2013,2030,18,Southeast,1823,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,200 E 104TH ST,NaN,"(33.942, -118.2717)"
3,101822289,11/11/2010,11/10/2010,1800,18,Southeast,1803,510,VEHICLE - STOLEN,NaN,...,NaN,IC,Invest Cont,510.0,NaN,NaN,NaN,88TH,WALL,"(33.9572, -118.2717)"
4,42104479,01/11/2014,01/04/2014,2300,21,Topanga,2133,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329,...,NaN,IC,Invest Cont,745.0,NaN,NaN,NaN,7200 CIRRUS WY,NaN,"(34.2009, -118.6369)"


In [3]:
data.columns

Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Area Name', 'Reporting District', 'Crime Code',
       'Crime Code Description', 'MO Codes', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code', 'Premise Description',
       'Weapon Used Code', 'Weapon Description', 'Status Code',
       'Status Description', 'Crime Code 1', 'Crime Code 2', 'Crime Code 3',
       'Crime Code 4', 'Address', 'Cross Street', 'Location '],
      dtype='object')

In [4]:
len(data)

1584316

In [5]:
# Number of null values for each column
data.isnull().sum()

DR Number                       0
Date Reported                   0
Date Occurred                   0
Time Occurred                   0
Area ID                         0
Area Name                       0
Reporting District              0
Crime Code                      0
Crime Code Description        412
MO Codes                   171759
Victim Age                 128659
Victim Sex                 145199
Victim Descent             145232
Premise Code                   76
Premise Description          2751
Weapon Used Code          1059559
Weapon Description        1059560
Status Code                     2
Status Description              0
Crime Code 1                    7
Crime Code 2              1484319
Crime Code 3              1582133
Crime Code 4              1584247
Address                         0
Cross Street              1321583
Location                        9
dtype: int64

## 1.1 Analyze Columns and Convert them to numerical representations

In [6]:
# Prepare data by removing descriptive columns and other unimportant columns.
# Removed MO Codes because it contains a list of codes which makes it hard to analyze.
# I don't see address and cross street being particularily useful.
# Not sure what Crime Codes 1 - 4 are so we can remove them for now.
# We will use the Date Occured instead of Date reported.
data2 = data[['DR Number', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Reporting District', 'Crime Code', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code',
       'Weapon Used Code', 'Status Code', 'Location ']]

### 1.1.1 Victim Sex

In [7]:
print(data2['Victim Sex'].unique())
print(data2.groupby('Victim Sex')['DR Number'].nunique())
print('Null:', data2['Victim Sex'].isnull().sum())

['F' nan 'M' 'H' 'X' '-']
Victim Sex
-         1
F    675402
H        53
M    739581
X     24080
Name: DR Number, dtype: int64
Null: 145199


In [8]:
# Turn Victim Sex into 0 for male, 1 for female, and 2 for other
data2['Victim Sex'].loc[data2['Victim Sex'] == 'M'] = 0
data2['Victim Sex'].loc[data2['Victim Sex'] == 'F'] = 1
data2['Victim Sex'].loc[data2['Victim Sex'].isna()] = 2
data2['Victim Sex'].loc[data2['Victim Sex'] == '-'] = 2
data2['Victim Sex'].loc[data2['Victim Sex'] == 'H'] = 2
data2['Victim Sex'].loc[data2['Victim Sex'] == 'X'] = 2

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-pa

In [9]:
data2.head()

,DR Number,Date Occurred,Time Occurred,Area ID,Reporting District,Crime Code,Victim Age,Victim Sex,Victim Descent,Premise Code,Weapon Used Code,Status Code,Location
0,1208575,03/11/2013,1800,12,1241,626,30.0,1,W,502.0,400.0,AO,"(33.9829, -118.3338)"
1,102005556,01/22/2010,2300,20,2071,510,NaN,2,NaN,101.0,NaN,IC,"(34.0454, -118.3157)"
2,418,03/18/2013,2030,18,1823,510,12.0,2,NaN,101.0,NaN,IC,"(33.942, -118.2717)"
3,101822289,11/10/2010,1800,18,1803,510,NaN,2,NaN,101.0,NaN,IC,"(33.9572, -118.2717)"
4,42104479,01/04/2014,2300,21,2133,745,84.0,0,W,501.0,NaN,IC,"(34.2009, -118.6369)"


In [10]:
# check that it worked
print(data2['Victim Sex'].unique())

[1 2 0]


### 1.1.2 Victim Descent

In [11]:
# Pulled from: https://ambarishg.github.io/books/LittleBookDataViz/victim-descent.html
VictimDescentAbbr = ("A","B","C","D","F","G","H","I","J","K","L","O","P","S","U","V","W","X","Z")
VictimDescentDescription = ("Other Asian","Black",
                             "Chinese","Cambodian","Filipino",
                             "Guamanian","Hispanic/Latin/Mexican",
                             "American Indian/Alaskan Native",
                             "Japanese","Korean","Laotian ",
                             "Other","Pacific Islander",
                             "Samoan","Hawaiian","Vietnamese",
                             "White","Unknown","AsianIndian")

In [12]:
print(data2['Victim Descent'].unique())
print(data2.groupby('Victim Descent')['DR Number'].nunique())
print('Null:', data2['Victim Descent'].isnull().sum())

['W' nan 'O' 'H' 'B' 'A' 'X' 'F' 'I' 'K' 'G' 'C' 'J' 'Z' 'P' 'V' 'U' 'L'
 'S' 'D' '-']
Victim Descent
-         1
A     37143
B    255056
C       618
D        15
F      1885
G        61
H    549515
I       663
J       237
K      7136
L        10
O    152776
P       276
S        24
U       137
V        86
W    391855
X     41535
Z        55
Name: DR Number, dtype: int64
Null: 145232


In [13]:
# The dictionary for the descent codes
descent_dict = [[0, None, None]]
for i in range(len(VictimDescentAbbr)):
    descent_dict.append([i+1, VictimDescentAbbr[i], VictimDescentDescription[i]])
descent_dict

[[0, None, None],
 [1, 'A', 'Other Asian'],
 [2, 'B', 'Black'],
 [3, 'C', 'Chinese'],
 [4, 'D', 'Cambodian'],
 [5, 'F', 'Filipino'],
 [6, 'G', 'Guamanian'],
 [7, 'H', 'Hispanic/Latin/Mexican'],
 [8, 'I', 'American Indian/Alaskan Native'],
 [9, 'J', 'Japanese'],
 [10, 'K', 'Korean'],
 [11, 'L', 'Laotian '],
 [12, 'O', 'Other'],
 [13, 'P', 'Pacific Islander'],
 [14, 'S', 'Samoan'],
 [15, 'U', 'Hawaiian'],
 [16, 'V', 'Vietnamese'],
 [17, 'W', 'White'],
 [18, 'X', 'Unknown'],
 [19, 'Z', 'AsianIndian']]

In [14]:
# Assign each letter a number, 'nan' and '-' should get 0

data2['Victim Descent'].loc[data2['Victim Descent'] == '-'] = 0
data2['Victim Descent'].loc[data2['Victim Descent'].isna()] = 0

for i in range(len(VictimDescentAbbr)):
    data2['Victim Descent'].loc[data2['Victim Descent'] == VictimDescentAbbr[i]] = i + 1

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [15]:
# Check to see that it worked
print(data2['Victim Descent'].unique())

[17  0 12  7  2  1 18  5  8 10  6  3  9 19 13 16 15 11 14  4]


### 1.1.3 Victim Age

In [16]:
print(data2['Victim Age'].unique())
#print(data2.groupby('Victim Age')['DR Number'].nunique())
#print('Null:', data2['Victim Age'].isnull().sum())

[30. nan 12. 84. 49. 27. 42. 29. 19. 23. 39. 18. 25. 34. 37. 50. 20. 32.
 24. 48. 45. 60. 40. 46. 22. 53. 36. 57. 56. 26. 52. 33. 21. 59. 28. 35.
 51. 41. 11. 10. 31. 43. 38. 73. 58. 70. 62. 55. 13. 66. 15. 68. 61. 63.
 54. 69. 44. 47. 14. 64. 93. 72. 65. 92. 75. 88. 97. 82. 17. 16. 81. 74.
 67. 71. 83. 77. 78. 80. 76. 99. 89. 79. 86. 94. 90. 85. 95. 91. 87. 96.
 98.]


### 1.1.4 Location

In [17]:
data2['Location '].head()

0    (33.9829, -118.3338)
1    (34.0454, -118.3157)
2     (33.942, -118.2717)
3    (33.9572, -118.2717)
4    (34.2009, -118.6369)
Name: Location , dtype: object

In [74]:
# Very slow computation
# Split the location into two seperate columns, longitude and latitude
templist = []
for i in range(len(data2['Location '])):
    try:
        temp = data2['Location '][i].split(',')
        templist.append([float(temp[0][1:]), float(temp[1][:-1])])
    except AttributeError:
        templist.append([np.nan, np.nan])
        print('AttributeError on value:', data2['Location '][i])

templist

AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan
AttributeError on value: nan


[[33.9829, -118.3338],
 [34.0454, -118.3157],
 [33.942, -118.2717],
 [33.9572, -118.2717],
 [34.2009, -118.6369],
 [34.0591, -118.2412],
 [34.1211, -118.2048],
 [34.241, -118.3987],
 [34.3147, -118.4589],
 [34.2012, -118.4662],
 [34.2354, -118.4517],
 [34.0294, -118.2671],
 [33.9651, -118.2783],
 [33.9651, -118.2754],
 [34.2012, -118.416],
 [33.9456, -118.2652],
 [34.0431, -118.2536],
 [34.0503, -118.2504],
 [34.0352, -118.2583],
 [34.048, -118.2577],
 [34.045, -118.264],
 [34.0407, -118.268],
 [34.0423, -118.2452],
 [34.0517, -118.2456],
 [34.0435, -118.2471],
 [34.0395, -118.2656],
 [34.2139, -118.4793],
 [34.0581, -118.2371],
 [34.0688, -118.2333],
 [34.0482, -118.2472],
 [34.0423, -118.2452],
 [34.045, -118.264],
 [34.045, -118.264],
 [34.051, -118.248],
 [34.0632, -118.2408],
 [34.0467, -118.2427],
 [34.043, -118.2341],
 [34.0423, -118.2452],
 [34.0451, -118.2604],
 [34.0503, -118.2433],
 [34.255, -118.4103],
 [34.043, -118.2606],
 [34.1351, -118.2153],
 [34.064, -118.2375],
 [34.

In [75]:
data2['Longitude'] = [item[0] for item in templist]
data2['Latitude'] = [item[1] for item in templist]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [76]:
data2.head()

,DR Number,Date Reported,Date Occurred,Time Occurred,Area ID,Reporting District,Crime Code,Victim Age,Victim Sex,Victim Descent,Premise Code,Weapon Used Code,Status Code,Location,Longitude,Latitude
0,1208575,2013-03-14,2013-03-11,1800,12,1241,626,30.0,1,17,502.0,400.0,AO,"(33.9829, -118.3338)",33.9829,-118.3338
1,102005556,2010-01-25,2010-01-22,2300,20,2071,510,NaN,2,0,101.0,NaN,IC,"(34.0454, -118.3157)",34.0454,-118.3157
2,418,2013-03-19,2013-03-18,2030,18,1823,510,12.0,2,0,101.0,NaN,IC,"(33.942, -118.2717)",33.9420,-118.2717
3,101822289,2010-11-11,2010-11-10,1800,18,1803,510,NaN,2,0,101.0,NaN,IC,"(33.9572, -118.2717)",33.9572,-118.2717
4,42104479,2014-01-11,2014-01-04,2300,21,2133,745,84.0,0,17,501.0,NaN,IC,"(34.2009, -118.6369)",34.2009,-118.6369


### 1.1.5 Status Code (removed from final dataset)

In [18]:
print(data2['Status Code'].unique())
print(data2.groupby('Status Code')['DR Number'].nunique())
print('Null:', data2['Status Code'].isnull().sum())

['AO' 'IC' 'AA' 'JA' 'JO' nan 'CC' 'TH' '13' '19']
Status Code
13          1
19          1
AA     162424
AO     178175
CC         24
IC    1227180
JA      12619
JO       3889
TH          1
Name: DR Number, dtype: int64
Null: 2


In [19]:
print(data['Status Description'].unique())
print(data.groupby('Status Description')['DR Number'].nunique())
print('Null:', data['Status Description'].isnull().sum())

['Adult Other' 'Invest Cont' 'Adult Arrest' 'Juv Arrest' 'Juv Other' 'UNK']
Status Description
Adult Arrest     162424
Adult Other      178175
Invest Cont     1227180
Juv Arrest        12619
Juv Other          3889
UNK                  29
Name: DR Number, dtype: int64
Null: 0


### 1.1.6 Date Occurred

In [20]:
# Print number of nulls
print('Null:', data2['Date Occurred'].isnull().sum())

Null: 0


In [25]:
# convert to datetime
data2['Date Occurred'] = pd.to_datetime(data2['Date Occurred'], format='%m/%d/%Y')

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [26]:
# Seperate the date column into month, day, year
data2['Month'] = data2['Date Occurred'].dt.month
data2['Day'] = data2['Date Occurred'].dt.day
data2['Year'] = data2['Date Occurred'].dt.year

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

### 1.1.7 Weapon Used Code

In [57]:
weap_dict = make_dict(data, 'Weapon Used Code', 'Weapon Description')

In [58]:
weap_dict

[[],
 [400.0, 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)'],
 [500.0, 'UNKNOWN WEAPON/OTHER WEAPON'],
 [207.0, 'OTHER KNIFE'],
 [212.0, 'BOTTLE'],
 [511.0, 'VERBAL THREAT'],
 [213.0, 'CLEAVER'],
 [204.0, 'FOLDING KNIFE'],
 [301.0, 'BELT FLAILING INSTRUMENT/CHAIN'],
 [200.0, 'KNIFE WITH BLADE 6INCHES OR LESS'],
 [302.0, 'BLUNT INSTRUMENT'],
 [515.0, 'PHYSICAL PRESENCE'],
 [501.0, 'BOMB THREAT'],
 [219.0, 'SCREWDRIVER'],
 [512.0, 'MACE/PEPPER SPRAY'],
 [509.0, 'ROPE/LIGATURE'],
 [205.0, 'KITCHEN KNIFE'],
 [101.0, 'REVOLVER'],
 [201.0, 'KNIFE WITH BLADE OVER 6 INCHES IN LENGTH'],
 [306.0, 'ROCK/THROWN OBJECT'],
 [102.0, 'HAND GUN'],
 [210.0, 'RAZOR BLADE'],
 [312.0, 'PIPE/METAL PIPE'],
 [218.0, 'OTHER CUTTING INSTRUMENT'],
 [106.0, 'UNKNOWN FIREARM'],
 [308.0, 'STICK'],
 [304.0, 'CLUB/BAT'],
 [109.0, 'SEMI-AUTOMATIC PISTOL'],
 [223.0, 'UNKNOWN TYPE CUTTING INSTRUMENT'],
 [309.0, 'BOARD'],
 [310.0, 'CONCRETE BLOCK/BRICK'],
 [114.0, 'AIR PISTOL/REVOLVER/RIFLE/BB GUN'],
 [216.0, 'SCISSOR

### 1.1.8 Crime Code

In [89]:
crime_dict = make_dict(data, 'Crime Code', 'Crime Code Description')

In [90]:
crime_dict

{626: 'INTIMATE PARTNER - SIMPLE ASSAULT',
 510: 'VEHICLE - STOLEN',
 745: 'VANDALISM - MISDEAMEANOR ($399 OR UNDER)',
 110: 'CRIMINAL HOMICIDE',
 668: 'EMBEZZLEMENT, GRAND THEFT ($950.01 & OVER)',
 440: 'THEFT PLAIN - PETTY ($950 & UNDER)',
 763: 'STALKING',
 442: 'SHOPLIFTING - PETTY THEFT ($950 & UNDER)',
 624: 'BATTERY - SIMPLE ASSAULT',
 670: 'EMBEZZLEMENT, PETTY THEFT ($950 & UNDER)',
 649: 'DOCUMENT FORGERY / STOLEN FELONY',
 480: 'BIKE - STOLEN',
 210: 'ROBBERY',
 821: 'SODOMY/SEXUAL CONTACT B/W PENIS OF ONE PERS TO ANUS OTH 0007=02',
 220: 'ATTEMPTED ROBBERY',
 437: 'RESISTING ARREST',
 888: 'TRESPASSING',
 341: 'THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LIVESTK,PROD0036',
 660: 'COUNTERFEIT',
 230: 'ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT',
 647: 'THROWING OBJECT AT MOVING VEHICLE',
 951: 'DEFRAUDING INNKEEPER/THEFT OF SERVICES, $400 & UNDER',
 930: 'CRIMINAL THREATS - NO WEAPON DISPLAYED',
 740: 'VANDALISM - FELONY ($400 & OVER, ALL CHURCH VANDALISMS) 0114',
 330: '

In [72]:
len(crime_dict)

138

In [98]:
pd.DataFrame(list(crime_dict.items()), columns=['Crime Code', 'Crime Description']).to_csv('../data/crime_codes.csv')

In [112]:
crimesTime = data['Time Occurred']
crimesTimeName = []
mornCount = 0
afterCount = 0
evenCount = 0
nightCount = 0
#set each time to morning, afternoon, evening or night 

for i in crimesTime:
    if i in range(500, 1159):
        crimesTimeName.append("Morning")
        mornCount += 1
    elif i in range(1200,1659):
        crimesTimeName.append("Afternoon")
        afterCount += 1
    elif i in range(1700, 2059):
        crimesTimeName.append("Evening")
        evenCount += 1
    else:
        crimesTimeName.append("Night")
        nightCount += 1
    
data['Time of Day'] = crimesTimeName

print(len(data))
print(len(crimesTime))

print('Crimes committed in the morning:',mornCount)
print('Crimes committed in the afternoon:',afterCount)
print('Crimes committed in the evening:',evenCount)
print('Crimes committed in the night:',nightCount)



1584316
1584316
Crimes committed in the morning: 343447
Crimes committed in the afternoon: 436065
Crimes committed in the evening: 363830
Crimes committed in the night: 440974


In [109]:
len(data['Crime Code'])
len(data)

tempdf = data['Time Occurred']
templist = []

for i in tempdf:
    if 500 >= i >= 1159:
        templist.append(0)
    elif 1200 >= i >= 1659:
        templist.append(1)
    elif 1700 >= i >= 2059:
        templist.append(2)
    else:
        templist.append(3)

data['Time of Day'] = templist

print(len(templist))
print(len(data))
print(len(tempdf))

1584316
1584316
1584316


In [108]:
type(tempdf)

pandas.core.series.Series

### 1.1.9 Premise Code

In [51]:
premise_dict = make_dict(data, 'Premise Code', 'Premise Description')

In [52]:
premise_dict

[[],
 [502.0, 'MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)'],
 [101.0, 'STREET'],
 [501.0, 'SINGLE FAMILY DWELLING'],
 [108.0, 'PARKING LOT'],
 [203.0, 'OTHER BUSINESS'],
 [404.0, 'DEPARTMENT STORE'],
 [102.0, 'SIDEWALK'],
 [406.0, 'OTHER STORE'],
 [735.0, 'NIGHT CLUB (OPEN EVENINGS ONLY)'],
 [701.0, 'HOSPITAL'],
 [732.0, 'POST OFFICE'],
 [122.0, 'VEHICLE, PASSENGER/TRUCK'],
 [402.0, 'MARKET'],
 [201.0, 'JEWELRY STORE'],
 [210.0, 'RESTAURANT/FAST FOOD'],
 [710.0, 'OTHER PREMISE'],
 [716.0, 'THEATRE/MOVIE'],
 [504.0, 'OTHER RESIDENCE'],
 [511.0, 'PROJECT/TENEMENT/PUBLIC HOUSING'],
 [207.0, 'BAR/COCKTAIL/NIGHTCLUB'],
 [719.0, 'MEDICAL/DENTAL OFFICES'],
 [107.0, 'VACANT LOT'],
 [104.0, 'DRIVEWAY'],
 [726.0, 'POLICE FACILITY'],
 [110.0, 'FREEWAY'],
 [503.0, 'HOTEL'],
 [405.0, 'CLOTHING STORE'],
 [403.0, 'DRUG STORE'],
 [109.0, 'PARK/PLAYGROUND'],
 [248.0, 'CELL PHONE STORE'],
 [717.0, 'HEALTH SPA/GYM'],
 [116.0, 'OTHER/OUTSIDE'],
 [407.0, 'HARDWARE/BUILDING SUPPLY'],
 [103.0, 'ALLEY'],
 [

### 1.1.10 Area ID

In [61]:
area_dict = make_dict(data, 'Area ID', 'Area Name')

In [62]:
area_dict

[[],
 [12, '77th Street'],
 [20, 'Olympic'],
 [18, 'Southeast'],
 [21, 'Topanga'],
 [1, 'Central'],
 [11, 'Northeast'],
 [16, 'Foothill'],
 [19, 'Mission'],
 [9, 'Van Nuys'],
 [13, 'Newton'],
 [15, 'N Hollywood'],
 [10, 'West Valley'],
 [14, 'Pacific'],
 [17, 'Devonshire'],
 [2, 'Rampart'],
 [3, 'Southwest'],
 [4, 'Hollenbeck'],
 [5, 'Harbor'],
 [6, 'Hollywood'],
 [8, 'West LA'],
 [7, 'Wilshire']]

# Make Dict Function

In [80]:
def make_dict(data, code, desc):    
    codes = data[code].unique()
    d = {}
    for i in codes:
        temp = data[data[code] == i]
        if not temp.empty:
            d.update({i : temp[desc].iloc[0]})
    return d

## 1.3 Compute Correlation Matrix

In [84]:
'''
Compute Correlation Matrix

Columns with High Correlation:
* Note: ignore crime codes 1 - 4
    99% : DR Number and Year
    99% : Area ID and Reporting District
    43% : Crime Code and Weapon Used Code
    99% : Crime Code and Crime Code 1
    17% : Victim Age and Premise Code
    -11% : Victim Age and Crime Code 4
    31% : Victim Age and Victim Sex
    17% : Victim Age and Victim Descent
    32% : Victim Sex and Victim Descent
    12% : Victim Sex and Premise Code
    19% : Victim Sex and Weapon Used Code
    15% : Victim Descent and Premise Code
    22% : Premise Code and Weapon Used Code
    17% : Premise Code and Crime Code 4
    43% : Weapon Used Code and Crime Code 1
    12% : Crime Code 1 and Crime Code 3
    26% : Crime Code 2 and Crime Code 3
    27% : Crime Code 2 and Crime Code 4
    51% : Crime Code 3 and Crime Code 4
'''
print(data2.columns)
data2.corr()

Index(['DR Number', 'Date Reported', 'Date Occurred', 'Time Occurred',
       'Area ID', 'Reporting District', 'Crime Code', 'Victim Age',
       'Victim Sex', 'Victim Descent', 'Premise Code', 'Weapon Used Code',
       'Status Code', 'Location ', 'Longitude', 'Latitude', 'Month', 'Day',
       'Year'],
      dtype='object')


,DR Number,Time Occurred,Area ID,Reporting District,Crime Code,Victim Age,Victim Sex,Victim Descent,Premise Code,Weapon Used Code,Longitude,Latitude,Month,Day,Year
DR Number,1.000000,-0.007390,0.016191,0.016248,-0.009166,0.007397,0.029931,0.009186,-0.015414,0.000441,-0.073152,0.073512,-0.066488,-0.003282,0.990360
Time Occurred,-0.007390,1.000000,0.011809,0.012023,0.017712,-0.049945,0.035793,-0.026228,-0.095394,-0.002821,-0.001711,0.002018,0.004076,0.023921,0.001187
Area ID,0.016191,0.011809,1.000000,0.998998,-0.004587,0.009835,0.010270,0.026257,0.014475,-0.000490,0.019657,-0.004529,-0.002669,-0.004693,-0.011645
Reporting District,0.016248,0.012023,0.998998,1.000000,-0.004624,0.010262,0.010168,0.025982,0.014411,-0.000368,0.019397,-0.004453,-0.002571,-0.004631,-0.011553
Crime Code,-0.009166,0.017712,-0.004587,-0.004624,1.000000,-0.034478,0.076948,-0.024452,0.098754,0.432260,0.000083,0.000468,-0.003484,-0.006296,-0.008273
Victim Age,0.007397,-0.049945,0.009835,0.010262,-0.034478,1.000000,-0.311614,0.270187,0.167376,0.074359,-0.000926,0.001905,0.000373,-0.012514,0.003336
Victim Sex,0.029931,0.035793,0.010270,0.010168,0.076948,-0.311614,1.000000,-0.322066,-0.118282,0.185867,-0.003341,0.001501,0.002633,0.000329,0.030298
Victim Descent,0.009186,-0.026228,0.026257,0.025982,-0.024452,0.270187,-0.322066,1.000000,0.151079,0.064199,0.001616,0.004579,-0.002049,-0.001794,0.006586
Premise Code,-0.015414,-0.095394,0.014475,0.014411,0.098754,0.167376,-0.118282,0.151079,1.000000,0.216194,0.001442,-0.000655,-0.007148,-0.026643,-0.027087
Weapon Used Code,0.000441,-0.002821,-0.000490,-0.000368,0.432260,0.074359,0.185867,0.064199,0.216194,1.000000,0.000776,0.000791,-0.004215,-0.011911,-0.002589


In [85]:
# remove status code, seems useless
# remove location and Date Occurred
data3 = data2[['DR Number', 'Month', 'Day', 'Year', 'Time Occurred',
       'Area ID', 'Reporting District', 'Crime Code', 'Victim Age', 'Victim Sex',
       'Victim Descent', 'Premise Code',
       'Weapon Used Code', 'Longitude', 'Latitude']]

In [86]:
data3.to_csv('../data/Crime_Data_2010_2017_clean.csv')

In [ ]:
# Remove Nulls
#data2 = data2.loc[data2['Location '].notnull()]